#### This notebooks add additional feature to ride_data.

In [1]:
import duckdb

jan_parquet = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\raw\ride_data_parquet\fhvhv_tripdata_2024-01.parquet"
taxi_zone = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\taxi_zones.parquet"

In [4]:
# Print taxi_Zone data
query = f"SELECT * FROM '{taxi_zone}'"
df = duckdb.query(query).to_df()
print(df.head(10))

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   
5         6    0.150491    0.000606  Arrochar/Fort Wadsworth           6   
6         7    0.107417    0.000390                  Astoria           7   
7         8    0.027591    0.000027             Astoria Park           8   
8         9    0.099784    0.000338               Auburndale           9   
9        10    0.099839    0.000436             Baisley Park          10   

         borough                                           geometry  \
0            EWR  [1, 3, 0, 0, 0, 1, 0, 0, 0, 232, 0, 0, 0, 136,...   
1         Queens  [1,

In [5]:
# Print ride data
query = f"SELECT * FROM '{jan_parquet}'"
df = duckdb.query(query).to_df()
print(df.head(10))

   hvfhs_license_num dispatching_base_num originating_base_num  \
0             HV0003               B03404               B03404   
1             HV0003               B03404               B03404   
2             HV0003               B03404               B03404   
3             HV0003               B03404               B03404   
4             HV0003               B03404               B03404   
..               ...                  ...                  ...   
95            HV0005               B03406                 None   
96            HV0003               B03404               B03404   
97            HV0003               B03404               B03404   
98            HV0003               B03404               B03404   
99            HV0003               B03404               B03404   

      request_datetime   on_scene_datetime     pickup_datetime  \
0  2024-01-01 00:21:47 2024-01-01 00:25:06 2024-01-01 00:28:08   
1  2024-01-01 00:10:56 2024-01-01 00:11:08 2024-01-01 00:12:53   
2  2024-0

In [9]:
# Drop the existing table before creating a new one
duckdb.query("DROP TABLE IF EXISTS cleaned_ride_data")

# Create the cleaned table
query = f"""
CREATE TABLE cleaned_ride_data AS 
SELECT request_datetime, pickup_datetime, dropoff_datetime, PULocationID, 
DOLocationID, trip_miles, trip_time, base_passenger_fare, 
congestion_surcharge, driver_pay
FROM '{jan_parquet}';
"""
duckdb.query(query)

# Save cleaned data as a new Parquet file
duckdb.query("COPY cleaned_ride_data TO 'C:/Users/FuQuan/Desktop/Ride-Hailing/Ride-Hailing-VRP-Problem/data/processed/ride_data/cleaned_ride_data.parquet' (FORMAT PARQUET)")

print("Successfully cleaned data and saved to 'C:/Users/FuQuan/Desktop/ride-hailing/data/processed/cleaned_ride_data.parquet'")

Successfully cleaned data and saved to 'C:/Users/FuQuan/Desktop/ride-hailing/data/processed/cleaned_ride_data.parquet'


In [10]:
test_parquet = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\ride_data\cleaned_ride_data.parquet"
query = f"SELECT * FROM '{test_parquet}' WHERE trip_miles > 5"
df = duckdb.query(query).to_df()
print(df.head())

     request_datetime     pickup_datetime    dropoff_datetime  PULocationID  \
0 2024-01-01 00:03:47 2024-01-01 00:06:15 2024-01-01 00:27:53           255   
1 2024-01-01 00:22:51 2024-01-01 00:29:47 2024-01-01 00:50:08            95   
2 2024-01-01 00:29:43 2024-01-01 00:37:50 2024-01-01 00:58:44           195   
3 2024-01-01 00:08:37 2024-01-01 00:11:17 2024-01-01 00:27:44           229   
4 2024-01-01 00:42:04 2024-01-01 00:53:16 2024-01-01 01:41:05           249   

   DOLocationID  trip_miles  trip_time  base_passenger_fare  \
0            95       7.020       1298                32.16   
1           212      11.330       1221                45.83   
2           112       7.370       1254                37.68   
3            87       5.477        987                22.78   
4           188       7.643       2869                35.10   

   congestion_surcharge  driver_pay  
0                  0.00       24.35  
1                  0.00       30.98  
2                  0.00       50

In [12]:
import duckdb
import pandas as pd

# Define file paths
ride_data_parquet = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\ride_data\cleaned_ride_data.parquet"
taxi_zone_parquet = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\taxi_zones.parquet"
output_parquet = r"C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\ride_data\enriched_ride_data.parquet"

# Step 1: Load Taxi Zones Data into DuckDB
print("\n=== Step 1: Load Taxi Zones into DuckDB ===")
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute(f"CREATE OR REPLACE TABLE taxi_zones AS SELECT * FROM '{taxi_zone_parquet}'")
print("\nTaxi Zones successfully loaded.")

# Step 2: Load Ride Data into DuckDB
print("\n=== Step 2: Load Ride Data into DuckDB ===")
conn.execute(f"CREATE OR REPLACE TABLE ride_data AS SELECT * FROM '{ride_data_parquet}'")
print("\nRide Data successfully loaded.")

# Step 3: Enrich Ride Data by Joining with Taxi Zones
print("\n=== Step 3: Enrich Ride Data ===")
enrichment_query = """
SELECT ride.*,
       tz_pickup.centroid_lat AS pickup_lat,
       tz_pickup.centroid_lon AS pickup_lon,
       tz_dropoff.centroid_lat AS dropoff_lat,
       tz_dropoff.centroid_lon AS dropoff_lon,
       tz_pickup.nearest_node_id AS pickup_nearest_node,
       tz_dropoff.nearest_node_id AS dropoff_nearest_node,
       date_part('epoch', ride.dropoff_datetime - ride.pickup_datetime) AS trip_duration,
       date_part('epoch', ride.pickup_datetime - ride.request_datetime) AS wait_time,
       CASE WHEN date_part('epoch', ride.dropoff_datetime - ride.pickup_datetime) > 0
            THEN ride.trip_miles / (date_part('epoch', ride.dropoff_datetime - ride.pickup_datetime) / 3600)
            ELSE NULL END AS avg_speed
FROM ride_data AS ride
LEFT JOIN taxi_zones AS tz_pickup
    ON ride.PULocationID = tz_pickup.LocationID
LEFT JOIN taxi_zones AS tz_dropoff
    ON ride.DOLocationID = tz_dropoff.LocationID
WHERE tz_pickup.nearest_node_id IS NOT NULL 
  AND tz_dropoff.nearest_node_id IS NOT NULL
"""
enriched_ride_data = conn.execute(enrichment_query).df()

# Step 4: Save Enriched Ride Data to Parquet
print("\n=== Step 4: Save Enriched Ride Data ===")
enriched_ride_data.to_parquet(output_parquet, index=False)
print(f"\n✅ Enriched ride data successfully saved to {output_parquet}")



=== Step 1: Load Taxi Zones into DuckDB ===

Taxi Zones successfully loaded.

=== Step 2: Load Ride Data into DuckDB ===

Ride Data successfully loaded.

=== Step 3: Enrich Ride Data ===

=== Step 4: Save Enriched Ride Data ===

✅ Enriched ride data successfully saved to C:\Users\FuQuan\Desktop\Ride-Hailing\Ride-Hailing-VRP-Problem\data\processed\ride_data\enriched_ride_data.parquet


In [13]:
query = f"SELECT * FROM '{output_parquet}' WHERE trip_miles > 5"
df = duckdb.query(query).to_df()
print(df.head())

     request_datetime     pickup_datetime    dropoff_datetime  PULocationID  \
0 2024-01-01 03:23:13 2024-01-01 04:27:50 2024-01-01 04:55:58            37   
1 2024-01-01 04:04:46 2024-01-01 04:08:30 2024-01-01 04:33:39           208   
2 2024-01-01 04:45:53 2024-01-01 04:50:15 2024-01-01 05:13:42            39   
3 2024-01-01 04:11:36 2024-01-01 04:22:18 2024-01-01 04:43:35           186   
4 2024-01-01 04:00:45 2024-01-01 04:02:23 2024-01-01 04:20:00           163   

   DOLocationID  trip_miles  trip_time  base_passenger_fare  \
0            26       9.570       1688                34.66   
1           180      15.519       1509                39.23   
2           225       5.153       1407                29.98   
3           243       9.730       1277                49.50   
4           217       8.090       1057                33.57   

   congestion_surcharge  driver_pay  pickup_lat  pickup_lon  dropoff_lat  \
0                  0.00       32.66   40.694994  -73.922240    40.6309